In [1]:
import pandas as pd
essays_df = pd.read_csv('C:/Users/moura/Projetos/Colab-Essays/dados/mypersonality_final.csv', encoding='latin-1')
essays_df = essays_df[['#AUTHID','STATUS', 'cEXT', 'cNEU', 'cAGR', 'cCON', 'cOPN']]

In [2]:
import numpy as np
import nltk
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# 1. Carregar o dataset e pré-processar

essays_df = pd.read_csv("C:/Users/moura/Projetos/Colab-Essays/dados/essays.csv", encoding="latin-1")


# Dividir os ensaios em sentenças
essays_df['sentences'] = essays_df['TEXT'].apply(nltk.sent_tokenize)

# Converter 'y' e 'n' para '1' e '0'
label_columns = ['cEXT', 'cNEU', 'cAGR', 'cCON', 'cOPN']
binarizer = LabelBinarizer()
for col in label_columns:
    essays_df[col] = binarizer.fit_transform(essays_df[col])

c:\Users\moura\Anaconda3\envs\colab_essays\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# 2. Obter embeddings das sentenças usando SentenceTransformer

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
essays_df['embeddings'] = essays_df['sentences'].apply(lambda sentences: [model.encode(sentence) for sentence in sentences])

# 3. Agregar embeddings para representar o ensaio completo

essays_df['essay_embedding'] = essays_df['embeddings'].apply(lambda embeddings: np.mean(embeddings, axis=0))

# 4. Dividir os dados em conjuntos de treinamento e teste

X = np.array(essays_df['essay_embedding'].tolist())
y = essays_df[label_columns].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# 5. Construir e treinar um modelo LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation='sigmoid'))

model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Reshape data for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# Define the early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


# Train the model with the callback
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stop])


# 6. Avaliar o modelo no conjunto de teste

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


In [ ]:
import pickle

# Após treinar o modelo
model.fit(X_train, y_train)

# Salvar o modelo em um arquivo
with open("mypersonality_lstm.pkl", "wb") as file:
    pickle.dump(model, file)

